In [1]:
import torch
import numpy as np

In [2]:
device = torch.device('cuda')
device

device(type='cuda')

In [3]:
vocab_dict = {'m': 0,
 'ё': 1,
 'e': 2,
 'i': 3,
 'ж': 4,
 '‑': 5,
 's': 6,
 'я': 7,
 'р': 8,
 'м': 9,
 'н': 10,
 '«': 11,
 'й': 12,
 'g': 13,
 'т': 14,
 '–': 15,
 'k': 16,
 'z': 17,
 '—': 18,
 'з': 19,
 "'": 20,
 'a': 21,
 'д': 22,
 'л': 23,
 '»': 24,
 'ч': 25,
 'с': 26,
 'б': 27,
 'h': 28,
 'c': 29,
 '(': 30,
 'и': 31,
 'l': 32,
 'щ': 33,
 'ф': 34,
 'o': 35,
 'ш': 36,
 'у': 37,
 'х': 38,
 'г': 39,
 'ц': 40,
 '…': 41,
 'ы': 42,
 'b': 43,
 'x': 44,
 'о': 45,
 'э': 46,
 'ъ': 47,
 'p': 48,
 'а': 49,
 'п': 50,
 'ю': 51,
 '−': 52,
 'е': 53,
 'в': 54,
 'ь': 55,
 ' ': 56,
 'r': 57,
 't': 58,
 'к': 59,
 ')': 60,
 'f': 61,
 'n': 62}

In [4]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [5]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

65

In [6]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [7]:
# !wget -O model.zip "https://onedrive.live.com/download?cid=656045B4D0378BFD&resid=656045B4D0378BFD%2116560&authkey=AAURl6JS-nXydU8"
# !unzip -o model.zip -d model_folder

In [8]:
# 200 checkpoint
# !wget -O model.zip "https://onedrive.live.com/download?cid=656045B4D0378BFD&resid=656045B4D0378BFD%2116563&authkey=AJwTvcA4r6gLx3s"
# !unzip -o model.zip -d model_folder

In [9]:
#546 checkpoint
# !wget -O model.zip "https://onedrive.live.com/download?cid=656045B4D0378BFD&resid=656045B4D0378BFD%2116574&authkey=ANBDXT4UHx1JBGE"
# !unzip -o model.zip -d model_folder

In [10]:
#846 checkpoint
# !wget -O model.zip "https://onedrive.live.com/download?cid=656045B4D0378BFD&resid=656045B4D0378BFD%2116575&authkey=AMPpp7yNM2alUXs"
# !unzip -o model.zip -d model_folder

In [11]:
#1146 checkpoint
!wget -O model.zip "https://onedrive.live.com/download?cid=656045B4D0378BFD&resid=656045B4D0378BFD%2116576&authkey=ANSmIkXaqLUWX-E"
!unzip -o model.zip -d model_folder

--2022-12-18 19:59:56--  https://onedrive.live.com/download?cid=656045B4D0378BFD&resid=656045B4D0378BFD%2116576&authkey=ANSmIkXaqLUWX-E
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ulwn4w.db.files.1drv.com/y4mEmeTIadF3k3bI1eRH_MvokKhiEkFHwFfI1CX6r2NtLj0EubbrZsp0D9i-XnTRVZKBB_a8p5um6rrahmADKuIMSWsWy2mTozJDMtAitsYJFY-Jfe-419cmMZ7gViaFwTaMxhj-xzLlaDunoJcxXAIuxLa0sRr1mWE_4TszNsPzJj5q3f-iZ8KGCYs4k5gclhm-U-_bXiK-5eSFMWOzHoO-A/checkpoint-1146.zip?download&psid=1 [following]
--2022-12-18 19:59:57--  https://ulwn4w.db.files.1drv.com/y4mEmeTIadF3k3bI1eRH_MvokKhiEkFHwFfI1CX6r2NtLj0EubbrZsp0D9i-XnTRVZKBB_a8p5um6rrahmADKuIMSWsWy2mTozJDMtAitsYJFY-Jfe-419cmMZ7gViaFwTaMxhj-xzLlaDunoJcxXAIuxLa0sRr1mWE_4TszNsPzJj5q3f-iZ8KGCYs4k5gclhm-U-_bXiK-5eSFMWOzHoO-A/checkpoint-1146.zip?download&psid=1
Resolving ulwn4w.db.files.1drv.com (ulwn4w.db

In [12]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("bond005/sberdevices_golos_100h_farfield", split="train", streaming=True)
common_voice_test = load_dataset("bond005/sberdevices_golos_100h_farfield", split="test", streaming=True)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

In [13]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [14]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

In [15]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
#     batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    try:
        batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    except:
        batch["transcription"] = "й" #replace empty string
    return batch

In [16]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [17]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer("/kaggle/working/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [18]:
import IPython.display as ipd
import numpy as np
import random

In [19]:
# it = iter(common_voice_train)
# next(it)
# sample = next(it)

In [20]:
# ipd.Audio(data=sample["audio"]["array"], autoplay=True, rate=16000)

In [21]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
#         batch["labels"] = processor(batch["sentence"]).input_ids
        batch["labels"] = processor(batch["transcription"]).input_ids
        
    return batch

In [22]:
common_voice_train = common_voice_train.map(prepare_dataset)
common_voice_test = common_voice_test.map(prepare_dataset)

In [23]:
# it = iter(common_voice_train)
# next(it)
# sample = next(it)

In [24]:
# import torchaudio

# def change_speed(sample):
#     speed_factor= 1.0
#     sample_rate = 16000
    
#     sox_effects = [
#             ["speed", str(speed_factor)],
#             ["rate", str(sample_rate)],
#             ['norm'],
#         ]
#     transformed_audio, _ = torchaudio.sox_effects.apply_effects_tensor(
#             torch.unsqueeze(torch.from_numpy(sample['input_values']), 0), sample_rate, sox_effects)
    
#     noise_effect = [['synth', '0', 'whitenoise']]

#     noise, _ = torchaudio.sox_effects.apply_effects_tensor(
#                 transformed_audio, sample_rate, noise_effect)

        
#     return transformed_audio+noise/50

In [25]:
# ipd.Audio(data=change_speed(sample), autoplay=True, rate=16000)

In [26]:
import torch
import torchaudio

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None
          

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [27]:
def speed_transform(batch, p=0.5):
    
    sample_rate=16000
    speed_factor = random.choice([0.9, 1.0, 1.1])

#     if speed_factor == 1.0: # no change
#         return batch
    
    if random.random()<p:
        # change speed and resample to original rate:
        sox_effects = [
            ["speed", str(speed_factor)],
            ["rate", str(sample_rate)],
            ['norm'],
#             ['echo',  '0.8', '0.88', '100', '0.4']
#             ['reverb', "-w"]
        ]
        transformed_audio, _ = torchaudio.sox_effects.apply_effects_tensor(
            torch.unsqueeze(torch.from_numpy(batch["input_values"]), 0), sample_rate, sox_effects)
        
        noise_effect = [['synth', '0', 'whitenoise']]

        noise, _ = torchaudio.sox_effects.apply_effects_tensor(
                    transformed_audio, sample_rate, noise_effect)

        batch['input_values'] = torch.squeeze(transformed_audio+noise/50).numpy()

    return batch

In [28]:
common_voice_train = common_voice_train.map(speed_transform)

In [29]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [30]:
%%capture
!pip install jiwer

In [31]:
wer_metric = load_metric("wer")

In [32]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [33]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
#     "facebook/wav2vec2-base-100k-voxpopuli", 
#     '/kaggle/working/model_folder/model',
    '/kaggle/working/model_folder', 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [34]:
model.freeze_feature_extractor()

/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1619: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [35]:
model.gradient_checkpointing_enable()

In [36]:
%%capture

!pip install neptune-client

In [37]:
import neptune.new as neptune

In [38]:
import os

In [39]:
os.environ['NEPTUNE_API_TOKEN'] = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3NTBlYjM5ZC1lZGYwLTQ3NGMtODg3Ni1mY2NmOWY4OGQzN2QifQ=="
os.environ['NEPTUNE_PROJECT'] = 'aibabynin/hw-audio2'

In [40]:
optimizer =  torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)

In [41]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-basee-ru-demo",
  group_by_length=False,
  per_device_train_batch_size=25,
  gradient_accumulation_steps=16,
  evaluation_strategy="steps",
  num_train_epochs=300,
  fp16=True,
  save_steps=50,
  eval_steps=50,
  logging_steps=10,
#   learning_rate=3e-4,
#   warmup_steps=50,
  save_total_limit=1,
    max_steps=300,
    report_to='neptune'
)

In [42]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train.with_format("torch"),
    eval_dataset=common_voice_test.with_format("torch"),
#     train_dataset=common_voice_train,
#     eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
    optimizers=(optimizer, scheduler)
)

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [43]:
#with augmentations
trainer.train()

***** Running training *****
  Num examples = 240000
  Num Epochs = 9223372036854775807
  Instantaneous batch size per device = 25
  Total train batch size (w. parallel, distributed & accumulation) = 800
  Gradient Accumulation steps = 16
  Total optimization steps = 300
/opt/conda/lib/python3.7/site-packages/transformers/integrations.py:928: NeptuneDeprecationWarning: `init` is deprecated, use `init_run` instead. We'll end support of it in `neptune-client==1.0.0`.
  run=os.getenv("NEPTUNE_RUN_ID", None),
/opt/conda/lib/python3.7/site-packages/neptune/new/internal/init/__init__.py:43: NeptuneDeprecationWarning: Parameter `run` is deprecated, use `with_id` instead. We'll end support of it in `neptune-client==1.0.0`.
  return init_run(*args, **kwargs)


https://app.neptune.ai/aibabynin/hw-audio2/e/HWAUD-14
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


/opt/conda/lib/python3.7/site-packages/neptune/new/attributes/attribute.py:64: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` instead.
  return self.assign(value, wait)
The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: audio, transcription. If audio, transcription are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Wer
50,0.510800,0.631781,0.675057
100,0.492600,0.643725,0.668376
150,0.445400,0.647678,0.674699
200,0.451100,0.639618,0.661935
250,0.411000,0.647588,0.673744
300,0.477000,0.631322,0.669689


***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: audio, transcription. If audio, transcription are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
Saving model checkpoint to ./wav2vec2-basee-ru-demo/checkpoint-50
Configuration saved in ./wav2vec2-basee-ru-demo/checkpoint-50/config.json
Model weights saved in ./wav2vec2-basee-ru-demo/checkpoint-50/pytorch_model.bin
Feature extractor saved in ./wav2vec2-basee-ru-demo/checkpoint-50/preprocessor_config.json
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 16
The follow

TrainOutput(global_step=300, training_loss=0.4725435733795166, metrics={'train_runtime': 9900.4105, 'train_samples_per_second': 24.241, 'train_steps_per_second': 0.03, 'total_flos': 9.124907585058109e+18, 'train_loss': 0.4725435733795166, 'epoch': 13.0})

In [44]:
!tar -zcvf model_1712_1446_steps.tar.gz wav2vec2-basee-ru-demo

wav2vec2-basee-ru-demo/
wav2vec2-basee-ru-demo/checkpoint-300/
wav2vec2-basee-ru-demo/checkpoint-300/scaler.pt
wav2vec2-basee-ru-demo/checkpoint-300/scheduler.pt
wav2vec2-basee-ru-demo/checkpoint-300/training_args.bin
wav2vec2-basee-ru-demo/checkpoint-300/trainer_state.json
wav2vec2-basee-ru-demo/checkpoint-300/pytorch_model.bin
wav2vec2-basee-ru-demo/checkpoint-300/config.json
wav2vec2-basee-ru-demo/checkpoint-300/rng_state.pth
wav2vec2-basee-ru-demo/checkpoint-300/optimizer.pt
wav2vec2-basee-ru-demo/checkpoint-300/preprocessor_config.json


In [45]:
from IPython.display import FileLink

In [46]:
FileLink(r'model_1712_1446_steps.tar.gz')

/kaggle/working/model_1712_1446_steps.tar.gz

In [47]:
import gc

In [48]:
gc.collect()

97